The particle dynamics in a general velocity field $ \mathbf{u}(\mathbf{x}, t) = \begin{pmatrix} u(\mathbf{x}, t) \\ v(\mathbf{x}, t) \\  w(\mathbf{x}, t) \end{pmatrix} $. This function evaluates the velocity field $ \mathbf{u}(\mathbf{x}, t) $, at point $ \mathbf{x} $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (3, NxNyNz) | $ \mathbf{x} $ |
| X | array (NY, NX, NZ) | X-meshgrid|
| Y | array (NY, NX, NZ) | Y-meshgrid|
| Z | array (NY, NX, NZ) | Y-meshgrid|
| Interpolant_u | RectangularGridInterpolator | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | RectangularGridInterpolator | Interpolant object for $ v(\mathbf{x}, t)  $ |
| Interpolant_w | RectangularGridInterpolator | Interpolant object for $ w(\mathbf{x}, t)  $ |
| periodic | list (4,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z <br /> periodic[3]: periodicity in time |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| vel | array (3, NxNyNz) | $ \begin{pmatrix} u(\mathbf{x}, t) \\ v(\mathbf{x}, t) \\ w(\mathbf{x}, t) \end{pmatrix} $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

In [3]:
def velocity(t, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady):
    
    x_scaled = x.copy()
    
    # check if periodic in x
    if periodic[0]:
        
        x_scaled[0,:] = x_scaled[0,:]%(X[0, -1, 0]-X[0, 0, 0])
    
    # check if periodic in y
    if periodic[1]:
        
        x_scaled[1,:] = x_scaled[1,:]%(Y[-1, 0, 0]-Y[0, 0, 0])
        
    # check if periodic in z
    if periodic[2]:
        
        x_scaled[2,:] = x_scaled[2,:]%(Z[0, 0, -1]-Z[0, 0, 0])
    
    x_scaled[0,:] = (x_scaled[0,:]-X[0, 0, 0])/(X[0, 1, 0]-X[0, 0, 0])
    x_scaled[1,:] = (x_scaled[1,:]-Y[0, 0, 0])/(Y[1, 0, 0]-Y[0, 0, 0])
    x_scaled[2,:] = (x_scaled[2,:]-Z[0, 0, 0])/(Z[0, 0, 1]-Z[0, 0, 0])
    
    x_scaled = x_scaled.transpose()
    
    x_swap = x_scaled[:, [1, 0, 2]]
    
    print(x_swap)
    
    if bool_unsteady:
    
        u = Interpolant_u(np.append(x_swap, t*np.ones(x.shape[0])))
        v = Interpolant_v(np.append(x_swap, t*np.ones(x.shape[0])))
        w = Interpolant_w(np.append(x_swap, t*np.ones(x.shape[0])))
        
    else:
        
        u = np.zeros((x_swap.shape[0], ))
        v = u.copy()
        w = u.copy()
        
        for i in range(x_swap.shape[0]):
            u[i] = Interpolant_u.ip(list(x_swap[i, :]))
            v[i] = Interpolant_v.ip(list(x_swap[i, :]))
            w[i] = Interpolant_w.ip(list(x_swap[i, :]))
        
    vel = np.array([u, v, w])
    
    return vel